<a href="https://colab.research.google.com/github/ionovm376/univercity/blob/main/Tasks/Ex_task_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Дополнительное задание №2.2. Замыкания. Декораторы. Итераторы. Генераторы**

**БАЗА:**

- **Замыкания** позволяют создавать функции с сохраняющимся состоянием. Это полезно для создания фабричных функций и функций с настраиваемым поведением.
- **Декораторы** позволяют модифицировать или расширять поведение функций без изменения их исходного кода.

---

## **I. Замыкания и декораторы**

### **Пункт №1**

Напишите две функции создания списка из чётных чисел от 0 до N (N – аргумент функции): \([0, 2, 4, ..., N]\).

- **Первая функция** должна использовать метод `append` для добавления элементов в список.
- **Вторая функция** должна использовать **генератор списков** (list comprehensions) для создания списка.

После этого, через **декоратор**, определите время работы этих функций.

In [ ]:
import time
def timer(func) -> "func":
    def wrapper(*args, **kwargs) -> "result":
        time_start = time.time()
        result = func(*args, **kwargs)
        time_finish = time.time()
        print(f"Время выполнение {func.__name__}: {time_finish - time_start:4f}")
        return result
    return wrapper
@timer
def app_lst(n: int) -> list[int]:
    lst = []
    for i in range(0, n + 1, 2): lst.append(i)
    return lst
@timer
def comprehensions_list(n: int) -> list[int]:
    return [i for i in range(0, n + 1, 2)]
print(app_lst(100))
print(comprehensions_list(100))

Время выполнение app_lst: 0.000006
[0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58, 60, 62, 64, 66, 68, 70, 72, 74, 76, 78, 80, 82, 84, 86, 88, 90, 92, 94, 96, 98, 100]
Время выполнение comprehensions_list: 0.000024
[0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58, 60, 62, 64, 66, 68, 70, 72, 74, 76, 78, 80, 82, 84, 86, 88, 90, 92, 94, 96, 98, 100]


---

### **Пункт №2**

Напишите **декоратор** для кэширования результатов работы функции, вычисляющей значение n-го числа [**ряда Фибоначчи**](https://ru.wikipedia.org/wiki/Числа_Фибоначчи).

То есть, при повторном вызове функции через декоратор уже имеющийся результат должен браться из кэша, а не вычисляться заново.

**Например:**

- При значении параметра `n = 5`, должна кэшироваться последовательность \([0, 1, 1, 2, 3, 5]\).
- Вызывая после этого целевую функцию через декоратор ещё раз с `n = 3`, результат \([0, 1, 1, 2]\) должен браться из кэша.
- Если последующее значение `n` больше предыдущего, например `n = 10`, вычисление должно продолжаться, начиная с закэшированной последовательности.

*Подсказка: используйте **замыкание** для хранения кэша внутри декоратора.*


In [ ]:
import functools
def fibonacci_cache(func):
    cache = [0, 1]
    @functools.wraps(func)
    def wrapper(n: int) -> int:
        if n < len(cache):
            return cache[n]
        result = func(n)
        cache.append(result)
        return cache[n]
    return wrapper
@fibonacci_cache
def fibonacci(n):
    if n in [0, 1]:
        return n
    return fibonacci(n - 1) + fibonacci(n - 2)
print(fibonacci(100))

354224848179261915075


---

### **Пункт №3**

Примените к функции из задания №2 сразу **два декоратора**:

1. **Декоратор**, определяющий время выполнения функции.
2. **Кэширующий декоратор** (из задания №2).

Сравните время работы функции с использованием кэширования и без него.


In [ ]:
import time
import functools
def measure_time(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        print(f"Время выполнения: {end_time - start_time:.6f} секунд")
        return result
    return wrapper
def cache_fib(func):
    cache = [0, 1]
    @functools.wraps(func)
    def wrapper(n):
        if n < len(cache):
            return cache[n]
        for i in range(len(cache), n + 1):
            next_value = cache[i-1] + cache[i-2]
            cache.append(next_value)
        return cache[n]
    return wrapper
@measure_time
@cache_fib
def fibonacci_cached(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fibonacci_cached(n-1) + fibonacci_cached(n-2)
@measure_time
def fibonacci_uncached(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fibonacci_uncached(n-1) + fibonacci_uncached(n-2)
print("С кэшированием:")
fibonacci_cached(30)
fibonacci_cached(5)
fibonacci_cached(35)
print("\nБез кэширования:")
fibonacci_uncached(30)

---

### **Пункт №4**

Создайте функцию **make_multiplier(n)**, которая принимает число **n** и возвращает функцию, умножающую переданное ей число на **n**.

**Пример использования:**

```python
def make_multiplier(n):
    # Ваш код

times3 = make_multiplier(3)
print(times3(5))  # Вывод: 15
```

In [ ]:
def make_multiplier(n: int) -> "func":
    def multiplier(x: int) -> int:
        return n * x
    return multiplier
times3 = make_multiplier(3)
print(times3(5))

15


---

### **Пункт №5**

Реализуйте функцию с замыканием, которая настраивает округление чисел до заданного количества знаков после запятой.

**Пример:**

```python
def rounder(n):
    # Ваш код

round_to_2 = rounder(2)
print(round_to_2(3.14159))  # Вывод: 3.14
```


In [ ]:
def rounder(n: int) -> "func":
    def rounder_n(x: float) -> float:
        return round(x, n)
    return rounder_n
round_to_2 = rounder(2)
print(round_to_2(3.14159))

3.14


---

### **Пункт №6**

Напишите **декоратор**, который измеряет время исполнения функции и выводит его на экран, только если время превышает определённый порог.

**Пример:**

```python
@time_threshold(threshold=0.5)
def long_running_function():
    # Долгий код

long_running_function()
# Выводится время выполнения только если оно больше 0.5 секунд
```

In [ ]:
import time
from functools import wraps
def time_threshold(threshold=0.5):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            start_time = time.perf_counter()
            result = func(*args, **kwargs)
            end_time = time.perf_counter()
            execution_time = end_time - start_time

            if execution_time > threshold:
                print(f"{execution_time:.4f} секунд")
            return result
        return wrapper
    return decorator
@time_threshold(threshold=0.5)
def long_running_function():
    time.sleep(0.7)
    return "Завершено"
@time_threshold(threshold=0.5)
def fast_function():
    time.sleep(0.2)
    return "Быстро завершено"
@time_threshold(threshold=1.0)
def calculate_sum(numbers):
    time.sleep(1.5)
    return sum(numbers)
result1 = long_running_function()
result2 = fast_function()
result3 = calculate_sum([1, 2, 3, 4, 5])
print(f"Результат 1: {result1}")
print(f"Результат 2: {result2}")
print(f"Результат 3: {result3}")

0.7003 секунд
1.5010 секунд
Результат 1: Завершено
Результат 2: Быстро завершено
Результат 3: 15


---

## **II. Итераторы и генераторы**

---

### **Пункт №1. Генератор строк фиксированной длины**

Напишите генератор `string_generator(char, times)`, который генерирует строки, состоящие из символа `char`, повторенного от 1 до `times` раз.

```python
# Пример использования:
for s in string_generator('*', 5):
    print(s)
# Вывод:
# *
# **
# ***
# ****
# *****
```



---

In [ ]:
def string_generator(char: str, times: int):
    for i in range(times + 1):
        yield char * i
for s in string_generator('*', 5):
    print(s)


*
**
***
****
*****


---

### **Пункт №2. Генератор бесконечной последовательности**

Создайте бесконечный генератор `infinite_sequence()`, который с каждым вызовом возвращает следующее число, начиная с 1.

```python
# Пример использования:
gen = infinite_sequence()
for _ in range(5):
    print(next(gen))
# Вывод:
# 1
# 2
# 3
# 4
# 5
```

---

In [ ]:
from typing import Generator
n = 1
def infinite_sequence() -> Generator[None, None, None]:
    global n
    while True:
        yield n
        n += 1
gen = infinite_sequence()
for _ in range(5):
    print(next(gen))

gen.close()

1
2
3
4
5


---

### **Пункт №3. Генератор комбинированных списков**

Создайте генератор `combined_lists(lst1, lst2)`, который попеременно возвращает элементы из `lst1` и `lst2`. Если длины списков неравны, генератор должен остановиться при исчерпании более короткого списка.

```python
# Пример использования:
for item in combined_lists([1, 2, 3], ['a', 'b', 'c', 'd']):
    print(item)
# Вывод:
# 1
# 'a'
# 2
# 'b'
# 3
# 'c'
```

---

In [ ]:
def combined_lists(lst1: int, lst2: int):
    min_length = min(len(lst1), len(lst2))
    for i in range(min_length):
        yield lst1[i]
        yield lst2[i]
for item in combined_lists([1, 2, 3], ['a', 'b', 'c', 'd']):
    print(item)

1
a
2
b
3
c


---

### **Пункт №4. Генератор перевернутой строки**

Напишите генератор `reverse_string(s)`, который при каждом вызове возвращает следующий символ строки `s` в обратном порядке.

```python
# Пример использования:
for char in reverse_string('hello'):
    print(char)
# Вывод:
# o
# l
# l
# e
# h
```

---

In [ ]:
def reverse_string(s: str):
    for i in s[::-1]: yield i
for char in reverse_string('hello'):
    print(char)

o
l
l
e
h


---

### **Пункт №5. Генератор степеней двойки**

Создайте генератор `powers_of_two(n)`, который возвращает степени двойки от 2^0 до 2^n.

```python
# Пример использования:
for num in powers_of_two(5):
    print(num)
# Вывод:
# 1  # 2^0
# 2  # 2^1
# 4  # 2^2
# 8  # 2^3
# 16 # 2^4
# 32 # 2^5
```

---

In [ ]:
from typing import Generator
def powers_of_two(n: int) -> Generator[int, None, None]:
    for i in range(0, n + 1): yield 2 ** i
for num in powers_of_two(5):
    print(num)

1
2
4
8
16
32


---

### **Пункт №6. Генератор чисел из строки**

Напишите генератор `number_extractor(s)`, который извлекает числа из заданной строки `s` и возвращает их как целые числа.

```python
# Пример использования:
for num in number_extractor('abc123def45gh6'):
    print(num)
# Вывод:
# 123
# 45
# 6
```

---

In [ ]:
from typing import Generator
import re
def number_extractoe(s: str) -> Generator[str, None, None]:
    for i in re.finditer(r'\d+', s):
        yield int(i.group())
for num in number_extractoe("abc123def45gh6"):
    print(num)

123
45
6


---